In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
# from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor as xgb

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, confusion_matrix, classification_report

from sklearn.feature_selection import VarianceThreshold

from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline  # Use the imblearn pipeline

%matplotlib inline

# Helper functions


In [42]:
# For the linear model
def Linear_regression(features_train, features_test, target_train, target_test):
    # Before using our data, we need to do feature scaling and we opt for the 'standardization' method of scaling.
    # The 'standardization' is avaliable thorugh the StandardScaler() method
    # Transformers help in batching tasks in a pipepline. In this case, the data is scaled and then a linear regression model is fitted on the scaled data.
    # We use a transformer that takes the regression model and the transformation method
    # The TransformedTargetRegressor does the transformation and when we do the prediction, it automatically does the inverse transformation (scaling) and returns the values
    bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    # categorical_cols = ['was_home']
    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', LinearRegression())
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())

    # TransformedTargetRegressor(
    #     LinearRegression(), transformer=StandardScaler())

    # fit the transofrmer on the train data
    model.fit(features_train, target_train)

    # With the model fitted, we can predict the total_points given the feature_train and feature_test set
    pred_train = model.predict(features_train)
    pred_test = model.predict(features_test)

    # Evaluate the performance of the model on both sets using the mean absolute error
    train_MAE = mean_absolute_error(target_train, pred_train)
    test_MAE = mean_absolute_error(target_test, pred_test)

    # Evaluate the performance of the model on both sets using the mean square error
    train_MSE = mean_squared_error(target_train, pred_train)
    test_MSE = mean_squared_error(target_test, pred_test)

    # Evaluate the performance of the model on both sets using the root mean square error
    train_RMSE = mean_squared_error(target_train, pred_train, squared=False)
    test_RMSE = mean_squared_error(target_test, pred_test, squared=False)

    # Get the score of the model or the coeeficient of determination i.e how much of the target value can be explained by the model.
    # In this case, 0.6 implies that 60% of the variations in the target value can be explained by the model and 40% is not explainable
    R2_train = model.score(features_train, target_train)
    R2_test = model.score(features_test, target_test)

    # If the test error significantly differs from the train error, then there is either overfitting or underfitting
    # RMSE, just like the squared loss function that it derives from, effectively penalizes larger errors more severely.
    print('Training set RMSE: {}'.format(train_RMSE))
    print('Test set RMSE: {}'.format(test_RMSE))

    print('Training set R2: {}'.format(R2_train))
    print('Test set R2: {}'.format(R2_test))

    # Carry out cross validation of the model.
    # The evaluation method is the root mean square error
    # The method expects a utility function (greater is better) and so the scoring function is the opposite of the the RMSE. Hence the -ve
    tree_rmses = -cross_val_score(model, features_train, target_train,
                                  scoring="neg_root_mean_squared_error", cv=10)

    return {'train_MAE': train_MAE, 'test_MAE': test_MAE, 'train_MSE': train_MSE, 'test_MSE': test_MSE, 'train_RMSE': train_RMSE, 'test_RMSE': test_RMSE, 'cv_rmse': tree_rmses.mean(), 'R2_train': R2_train, 'R2_test': R2_test}


# Decision Tree Model
def DecisionTreeRegression(features_train, features_test, target_train, target_test):
    # The DecisionTreeRegressor is passed as the model to the TransformedTreeRegressor together with the StandardScaler
    bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    # categorical_cols = ['was_home']
    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', DecisionTreeRegressor())
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())
    # TransformedTargetRegressor(
    #     DecisionTreeRegressor(), transformer=StandardScaler())
    model.fit(features_train, target_train)

    pred_train = model.predict(features_train)
    pred_test = model.predict(features_test)

    train_MAE = mean_absolute_error(target_train, pred_train)
    test_MAE = mean_absolute_error(target_test, pred_test)

    train_MSE = mean_squared_error(target_train, pred_train)
    test_MSE = mean_squared_error(target_test, pred_test)

    train_RMSE = mean_squared_error(target_train, pred_train, squared=False)
    test_RMSE = mean_squared_error(target_test, pred_test, squared=False)

    R2_train = model.score(features_train, target_train)
    R2_test = model.score(features_test, target_test)

    tree_rmses = -cross_val_score(model, features_train, target_train,
                                  scoring="neg_root_mean_squared_error", cv=10)

    return {'train_MAE': train_MAE, 'test_MAE': test_MAE, 'train_MSE': train_MSE, 'test_MSE': test_MSE,
            'train_RMSE': train_RMSE, 'test_RMSE': test_RMSE, 'cv_rmse': tree_rmses.mean(), 'R2_train': R2_train, 'R2_test': R2_test}


# RandomForestRegressor
def RandomForestRegression(features_train, features_test, target_train, target_test, hyperparameters):
    # RandomForestRegressor is an ensemble method
    # The TransformedTargetRegressor is passed the RandomForestRegressor model
    # The RandomForestRegressor is passed some hyper-parameters such as;
    # n_esimtaors: number of trees in the forest,
    # max_depth: the maximum depth of the tree,
    # criterion: the function to measure the quality of the split

    bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    # categorical_cols = ['was_home']
    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(
                        n_estimators=hyperparameters['n_estimators'],
                        max_depth=hyperparameters['max_depth'],
                        criterion=hyperparameters['criterion'], random_state=18
                        ),)
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())

    # model = TransformedTargetRegressor( transformer=StandardScaler())
    model.fit(features_train, target_train)

    pred_train = model.predict(features_train)
    pred_test = model.predict(features_test)

    train_MAE = mean_absolute_error(target_train, pred_train)
    test_MAE = mean_absolute_error(target_test, pred_test)

    train_MSE = mean_squared_error(target_train, pred_train)
    test_MSE = mean_squared_error(target_test, pred_test)

    train_RMSE = mean_squared_error(target_train, pred_train, squared=False)
    test_RMSE = mean_squared_error(target_test, pred_test, squared=False)

    R2_train = model.score(features_train, target_train)
    R2_test = model.score(features_test, target_test)

    tree_rmses = -cross_val_score(model, features_train, target_train,
                                  scoring="neg_root_mean_squared_error", cv=10)

    return {'train_MAE': train_MAE, 'test_MAE': test_MAE, 'train_MSE': train_MSE, 'test_MSE': test_MSE,
            'train_RMSE': train_RMSE, 'test_RMSE': test_RMSE, 'cv_rmse': tree_rmses.mean(), 'R2_train': R2_train, 'R2_test': R2_test}


def XGBoostRegression(features_train, features_test, target_train, target_test, hyperparameters):

    bool_cols = features_train.drop(columns=['was_home']).columns.tolist()
    # categorical_cols = ['was_home']
    bool_cols = features_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = features_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', xgb(learning_rate=hyperparameters["learning_rate"],
                    n_estimators=hyperparameters["n_estimators"],
                    max_depth=hyperparameters["max_depth"],
                    eval_metric='rmsle'),)
    ])

    model = TransformedTargetRegressor(regressor=pipeline, transformer=StandardScaler())

    model.fit(features_train, target_train)

    # =========================================================================
    # To use early_stopping_rounds:
    # "Validation metric needs to improve at least once in every
    # early_stopping_rounds round(s) to continue training."
    # =========================================================================
    # first perform a test/train split
    # from sklearn.model_selection import train_test_split

    # X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size = 0.2)
    # model.fit(X_train, y_train, early_stopping_rounds=6, eval_set=[(X_test, y_test)], verbose=False)

    # =========================================================================
    # use the model to predict the prices for the test data
    # =========================================================================
    # predictions = model.predict(goalkeepers_splits['feature_test'])

    pred_train = model.predict(features_train)
    pred_test = model.predict(features_test)

    train_MAE = mean_absolute_error(target_train, pred_train)
    test_MAE = mean_absolute_error(target_test, pred_test)

    train_MSE = mean_squared_error(target_train, pred_train)
    test_MSE = mean_squared_error(target_test, pred_test)

    train_RMSE = mean_squared_error(target_train, pred_train, squared=False)
    test_RMSE = mean_squared_error(target_test, pred_test, squared=False)

    R2_train = model.score(features_train, target_train)
    R2_test = model.score(features_test, target_test)

    tree_rmses = -cross_val_score(model, features_train, target_train,
                                  scoring="neg_root_mean_squared_error", cv=10)

    return {'train_MAE': train_MAE, 'test_MAE': test_MAE, 'train_MSE': train_MSE, 'test_MSE': test_MSE,
            'train_RMSE': train_RMSE, 'test_RMSE': test_RMSE, 'cv_rmse': tree_rmses.mean(), 'R2_train': R2_train, 'R2_test': R2_test}

def Logistic_regression(features_train, features_test, target_train, target_test):
    encoder = LabelEncoder()
    cs_train_ = encoder.fit_transform(target_train)
    cs_test_ = encoder.transform(target_test)

    # bool_cols = feats_train.drop(columns=['was_home']).columns.tolist()
    # categorical_cols = ['was_home']
    bool_cols = feats_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = feats_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False)),
        # ('to_dense', ToDense())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', BorderlineSMOTE(sampling_strategy='minority', random_state=42)),
        ('model', LogisticRegression(class_weight='balanced'))
    ])

    pipeline.fit(features_train, cs_train_)

    train_score = pipeline.score(features_train, cs_train_)
    test_score = pipeline.score(features_test, cs_test_)
    # Make predictions on the test set
    cs_pred = pipeline.predict(features_test)

    # Evaluate the model
    accuracy =  accuracy_score(cs_test_, cs_pred)

    conf_mat = confusion_matrix(cs_test_,cs_pred)

    class_report = classification_report(cs_test_, cs_pred)

    unique, counts = np.unique(cs_test_, return_counts=True)
    print("Class distribution:", dict(zip(unique, counts)))


    return {'train_score': train_score, 'test_score': test_score, 'accuracy': accuracy, 'conf_mat': conf_mat, 'class_report': class_report}


def Random_Forest_Classifier(features_train, features_test, target_train, target_test):
    encoder = LabelEncoder()
    cs_train_ = encoder.fit_transform(target_train)
    cs_test_ = encoder.transform(target_test)

    # bool_cols = feats_train.drop(columns=['was_home']).columns.tolist()
    # categorical_cols = ['was_home']
    bool_cols = feats_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_cols = feats_train.select_dtypes(include=['object', 'category']).columns.tolist()

    numerical_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('one_hot_encoder', OneHotEncoder(sparse=False)),
        # ('to_dense', ToDense())
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, bool_cols),
            ('cat', categorical_transformer, categorical_cols),
        ])

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', BorderlineSMOTE(sampling_strategy='auto', random_state=42)),  # Apply SMOTE to the data
        ('classifier', RandomForestClassifier(bootstrap = False, max_depth= None, max_features= 'sqrt', min_samples_leaf = 1, min_samples_split = 2, n_estimators = 100 ,class_weight='balanced', random_state=42))  # Random Forest Classifier
    ])


    pipeline.fit(features_train, cs_train_)

    train_score = pipeline.score(features_train, cs_train_)
    test_score = pipeline.score(features_test, cs_test_)
    # Make predictions on the test set
    cs_pred = pipeline.predict(features_test)

    # Evaluate the model
    accuracy =  accuracy_score(cs_test_, cs_pred)

    conf_mat = confusion_matrix(cs_test_,cs_pred)

    class_report = classification_report(cs_test_, cs_pred)

    unique, counts = np.unique(cs_test_, return_counts=True)


    return {'train_score': train_score, 'test_score': test_score, 'accuracy': accuracy, 'conf_mat': conf_mat, 'class_report': class_report}


def GridSearchParams(features_train, target_train):
    # Instatiate the model
    model = RandomForestRegressor()

    param_grid = {'n_estimators': [8, 10, 12, 14, 16, 18, 20]}

    # Define the possible values of the hyperparameter
    grid = {
        'n_estimators': [8, 10, 12, 14, 16, 18, 20, 200, 300, 400, 500],
        'max_features': ['sqrt', 'log2'],
        'max_depth': [4, 5, 6, 7, 8],
        'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
        'random_state': [18]
    }

    # Deine the model with cv=3 for a 3-fold cross validation
    # GridSearchCV has the best_estimator_ parameter that returns the  estimator
    # which gave highest score (or smallest loss if specified)

    grid_search = GridSearchCV(
        model, grid, cv=3, scoring='neg_root_mean_squared_error')
    grid_search.fit(features_train, target_train)

    # Get the best param combination
    print(grid_search.best_estimator_)

    return {'train_RMSE': train_RMSE, 'test_RMSE': test_RMSE, 'R2_train': R2_train, 'R2_test': R2_test}

In [43]:

# define a function that splits and returns features_train, features_test, target_train, target_test

def split_data(data):
    # Store the 'total_points' target in the 'player_target' variable
    # and the rest in the player_features variable
    player_target = data['pts_bps']
    player_features = data.drop("pts_bps", axis=1)

    # The train_test_split function splits the set into train and test sets while maintain the same data distribution over both sets.
    # It takes the feature and target sets and reutrns the respective train and test sets
    features_train, features_test, target_train, target_test = train_test_split(
        player_features, player_target, test_size=0.2)

    return {'feature_train': features_train, 'features_test': features_test, 'target_train': target_train, 'target_test': target_test}

# Get the data


In [59]:
data_22_23 = pd.read_csv('./data/joint/22-23/merged_player_data.csv').dropna()
data_23_24 = pd.read_csv('./data/joint/23-24/merged_player_data.csv').dropna()
data_24_25 = pd.read_csv('./data/joint/24-25/merged_player_data.csv').dropna()
data_tar = data_24_25[data_24_25['event']==20]
data_24_25_ = data_24_25[data_24_25['event'] != 20]

data = pd.concat([data_22_23, data_23_24, data_24_25_])
predicted = pd.DataFrame()


In [66]:
pd.read_csv('./data/joint/24-25/merged_extras_odds/Bruno Borges Fernandes.csv')

,assists_x,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,...,npg_5,npxG_5,xGChain_5,xGBuildup_5,xP_5,selected_5,pts_bps,whh,whd,wha
0,0,0,10,1,8.3,366,0.03,1.22,1.19,0.44,...,0.0,0.00,0.00,0.00,0.0,0.0,NaN,NaN,NaN,NaN
1,0,0,15,0,37.7,366,0.73,0.80,0.07,1.50,...,0.0,0.94,1.08,0.15,2.9,1728175.0,NaN,NaN,NaN,NaN
2,0,0,16,0,9.6,366,0.07,0.09,0.02,1.73,...,0.0,1.02,1.73,0.67,5.4,3504795.0,NaN,NaN,NaN,NaN
3,1,0,20,1,18.2,366,0.35,0.57,0.22,1.12,...,0.0,1.04,2.23,1.15,7.2,4850625.0,NaN,NaN,NaN,NaN
4,0,0,12,1,40.0,366,0.14,0.50,0.36,1.01,...,0.0,1.30,3.38,2.05,10.7,6029487.0,NaN,NaN,NaN,NaN
5,0,0,-7,0,1.2,366,0.02,0.02,0.00,1.99,...,0.0,1.76,4.19,2.45,14.0,7168882.0,NaN,NaN,NaN,NaN
6,0,0,14,1,5.3,366,0.02,0.06,0.04,0.42,...,0.0,0.83,3.28,2.48,13.1,6405125.0,NaN,NaN,NaN,NaN
7,1,0,26,0,26.2,366,0.05,0.17,0.12,0.86,...,0.0,0.78,2.63,1.96,12.8,5351187.0,NaN,NaN,NaN,NaN
8,0,0,17,0,50.8,366,0.28,0.74,0.46,2.98,...,0.0,1.01,3.02,1.79,13.0,4720589.0,NaN,NaN,NaN,NaN
9,0,3,41,0,50.6,366,0.10,1.37,1.27,1.11,...,0.0,1.22,3.66,1.48,11.5,4257741.0,NaN,NaN,NaN,NaN


In [58]:
data_tar['event'].unique()

array([], dtype=float64)

```
    Total Points – Bonus Points (tp-bp), Minutes, Yellow Cards, Red Cards, Expected Goals (xG), Expected Assists (xA), Non-penalty Expected Goals (npxG),
    Shots, Expected Goals Against, _Expected_goal_involvements_,  clean_sheets, ict_index, opponent_team, Expected Goals Buildup (xG Buildup), threat, value,
    Key Passes,


    _Games_,  Expected Goals Chain (xG Chain),  _Non-penalty Expected Goal Difference (npxGD)_, _Non-penalty Expected Goals Against (npxGA)_, Expected Points (xPts)
```

```js
    Total Points – Bonus Points (tp-bp)	for, mid, def, gk
    Minutes	for, mid, def, gk
    Yellow Cards	for, mid, def, gk
    Red Cards	for, mid, def, gk
    Expected Goals (xG)	for, mid, def, gk
    Expected Assists (xA)	for, mid, def, gk
    Non-penalty Expected Goals (npxG)	for, mid, def
    Games	for, mid, def, gk
    Shots	for, mid, def
    Key Passes	for, mid, def, gk
    Expected Goals Chain (xG Chain)	for, mid, def
    Expected Goals Buildup (xG Buildup)	for, mid, def
    Non-penalty Expected Goal Difference (npxGD)	def, gk
    Expected Goals Against	def, gk
    Non-penalty Expected Goals Against (npxGA)	def, gk
    Expected Points (xPts)	def, gk
    expected_goal_involvements	for, mid, def, gk
    clean_sheets	mid, def, gk
    ict_index	for, mid, def, gk
    opponent_team	for, mid, def, gk
    threat	for, mid, def, gk
    value	for, mid, def, gk

```
